In [79]:
import itertools
import math
import os
import gsd.hoomd
import hoomd
import numpy
import pandas as pd

In [80]:
N_particles = 108
rho = 0.8442
target_total_energy_per_particle = -2.1626
box_volume = N_particles/rho
L = box_volume**(1/3)

spacing = L / math.ceil(N_particles ** (1/3))

x = numpy.linspace(-L / 2, L / 2, math.ceil(N_particles ** (1/3)), endpoint=False)
position = list(itertools.product(x, repeat=3))

In [81]:
frame = gsd.hoomd.Frame()
frame.particles.N = N_particles
frame.particles.position = position[0:N_particles]
frame.particles.typeid = [0] * N_particles
frame.configuration.box = [L, L, L, 0, 0, 0]

In [82]:
frame.particles.types = ["A","B"]

# Set most particles as type A, one as type B
typeid = [0] * N_particles  # All type A initially
typeid[0] = 1  # Make first particle type B (active)
frame.particles.typeid = typeid

In [83]:
if os.path.exists("lattice.gsd"):
    os.remove("lattice.gsd")

# Now open and write to the file
with gsd.hoomd.open(name="lattice.gsd", mode="x") as f:
    f.append(frame)

In [84]:
cpu = hoomd.device.CPU()
simulation = hoomd.Simulation(device=cpu, seed=1)
simulation.create_state_from_gsd(filename="lattice.gsd")

In [85]:
# Create the integrator
integrator = hoomd.md.Integrator(dt=0.0001)

# Set up the neighbor list
cell = hoomd.md.nlist.Cell(buffer=0.019)  

In [86]:
lj = hoomd.md.pair.LJ(nlist=cell, mode='shift')
lj.params[("A", "A")] = dict(epsilon=1, sigma=1)
lj.params[("A", "B")] = dict(epsilon=1, sigma=1)  # Active-passive interaction
lj.params[("B", "B")] = dict(epsilon=1, sigma=1)  # Active-active (though only 1 particle)
lj.r_cut[("A", "A")] = 2.5
lj.r_cut[("A", "B")] = 2.5  # Add this line
lj.r_cut[("B", "B")] = 2.5  # Add this line
# Add the force to the integrator
integrator.forces.append(lj)

In [87]:
# Create active force
active_filter = hoomd.filter.Type(['B'])
active_force = hoomd.md.force.Active(filter=active_filter)

# Set active force magnitude (in local particle frame - always +x direction)
force_magnitude = 10  # Adjust as needed
active_force.active_force['B'] = (force_magnitude, 0.0, 0.0)
active_force.active_torque['B'] = (0.0, 0.0, 0.0)

# Add to integrator
integrator.forces.append(active_force)

rotational_diffusion_updater = active_force.create_diffusion_updater(
    trigger=hoomd.trigger.Periodic(10),  # Update every 10 steps
    rotational_diffusion=100 # Diffusion coefficient
)
simulation.operations.updaters.append(rotational_diffusion_updater)

In [88]:
nve = hoomd.md.methods.ConstantVolume(filter=hoomd.filter.All(), thermostat=None)
integrator.methods.append(nve)
simulation.operations.integrator = integrator  # Assign integrator to simulation

In [89]:
# Create a custom action class to log active particle info
class ActiveParticleLogger(hoomd.custom.Action):
    def __init__(self, filename):
        super().__init__()
        # Create the log file with a header
        with open(filename, "w") as f:
            f.write("timestep pos_x pos_y pos_z director_x director_y director_z quaternion_w quaternion_x quaternion_y quaternion_z\n")
        self.filename = filename
        
    def act(self, timestep):
        # Get current snapshot
        snapshot = self._state.get_snapshot()
        active_pos = snapshot.particles.position[0]  # Active particle position
        active_orient = snapshot.particles.orientation[0]  # Active particle orientation
        
        # Convert quaternion to director vector (local x-axis direction)
        q = active_orient
        director = numpy.array([
            1 - 2*(q[1]**2 + q[2]**2),
            2*(q[0]*q[1] + q[2]*q[3]),
            2*(q[0]*q[2] - q[1]*q[3])
        ])
        
        # Write to the log file
        with open(self.filename, "a") as f:
            f.write(f"{timestep} {active_pos[0]:.6f} {active_pos[1]:.6f} {active_pos[2]:.6f} "
                   f"{director[0]:.6f} {director[1]:.6f} {director[2]:.6f} "
                   f"{q[0]:.6f} {q[1]:.6f} {q[2]:.6f} {q[3]:.6f}\n")

# Create the active particle logger
active_logger = ActiveParticleLogger(filename="active_particle_log.txt")

# Create a custom writer with the logger
active_writer = hoomd.write.CustomWriter(
    action=active_logger,
    trigger=hoomd.trigger.Periodic(period=10)  # Log every 100 timesteps
)

# Add the writer to the simulation
simulation.operations.writers.append(active_writer)

In [90]:
# currently the velocities are 0
snapshot = simulation.state.get_snapshot()
snapshot.particles.velocity[0:5]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [91]:
simulation.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=1.5)

In [92]:
simulation.run(100000)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from matplotlib.animation import FuncAnimation, PillowWriter

# Show current working directory
current_dir = os.getcwd()
print(f"Working directory: {current_dir}")

print("Loading data...")
data = pd.read_csv('active_particle_log.txt', sep=r'\s+', engine='python')

# CONTROL FRAMES HERE - Change this number to use different amounts of data
MAX_FRAMES = 400  # Options: 100 (fast), 400 (good balance), 1000 (detailed), 2000+ (very long)
data = data.head(MAX_FRAMES)

print(f"Loaded {len(data)} frames from file (using first {MAX_FRAMES} frames)")

# Get coordinates
x = data['pos_x'].values
y = data['pos_y'].values  
z = data['pos_z'].values
dx = data['director_x'].values
dy = data['director_y'].values
dz = data['director_z'].values

# Create figure for animation
fig = plt.figure(figsize=(16, 8))

def animate_frame(frame):
    # Use every 2nd data point for smoother animation
    i = frame * 2
    if i >= len(data):
        i = len(data) - 1
    
    # Clear previous plots
    fig.clear()
    
    # Create subplots
    ax1 = fig.add_subplot(121)  # 2D plot
    ax2 = fig.add_subplot(122, projection='3d')  # 3D plot
    
    # 2D plot setup
    ax1.set_xlim(x.min()-0.15, x.max()+0.15)
    ax1.set_ylim(y.min()-0.15, y.max()+0.15)
    ax1.set_xlabel('X Position', fontsize=12)
    ax1.set_ylabel('Y Position', fontsize=12)
    ax1.set_title(f'2D View - Frame {i+1}/{len(data)}', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # 3D plot setup
    ax2.set_xlim(x.min()-0.15, x.max()+0.15)
    ax2.set_ylim(y.min()-0.15, y.max()+0.15)
    ax2.set_zlim(z.min()-0.15, z.max()+0.15)
    ax2.set_xlabel('X Position', fontsize=10)
    ax2.set_ylabel('Y Position', fontsize=10)
    ax2.set_zlabel('Z Position', fontsize=10)
    ax2.set_title(f'3D View - Timestep {data["timestep"].iloc[i]}', fontsize=14, fontweight='bold')
    
    # Plot trajectories
    if i > 0:
        # Full path (very faint)
        ax1.plot(x[:i+1], y[:i+1], color='lightblue', linewidth=0.8, alpha=0.3, label='Full path')
        ax2.plot(x[:i+1], y[:i+1], z[:i+1], color='lightblue', linewidth=0.8, alpha=0.3)
        
        # Recent trail (bright)
        trail_length = min(40, i+1)
        start_idx = max(0, i - trail_length + 1)
        if trail_length > 1:
            ax1.plot(x[start_idx:i+1], y[start_idx:i+1], color='blue', linewidth=4, alpha=0.8, label='Recent trail')
            ax2.plot(x[start_idx:i+1], y[start_idx:i+1], z[start_idx:i+1], color='blue', linewidth=4, alpha=0.8)
    
    # Current position
    ax1.scatter(x[i], y[i], color='red', s=120, zorder=5, label='Current position', edgecolors='darkred', linewidth=2)
    ax2.scatter(x[i], y[i], z[i], color='red', s=150, zorder=5, edgecolors='darkred', linewidth=2)
    
    # Director arrow in 3D
    arrow_scale = 0.25
    ax2.quiver(x[i], y[i], z[i], dx[i]*arrow_scale, dy[i]*arrow_scale, dz[i]*arrow_scale, 
               color='green', arrow_length_ratio=0.3, linewidth=3, alpha=0.9)
    
    # Director arrow in 2D (projected)
    ax1.arrow(x[i], y[i], dx[i]*arrow_scale*0.5, dy[i]*arrow_scale*0.5, 
              head_width=0.05, head_length=0.03, fc='green', ec='green', linewidth=2, alpha=0.9)
    
    # Add legend to 2D plot
    if i > 0:
        ax1.legend(loc='upper right', fontsize=10)
    
    # Add progress text
    progress = (i + 1) / len(data) * 100
    fig.suptitle(f'HOOMD Active Particle Simulation - Progress: {progress:.1f}%', 
                fontsize=16, fontweight='bold', y=0.95)
    
    plt.tight_layout()
    
    # Print progress
    if frame % 10 == 0:
        print(f"Rendering frame {frame+1}/{len(data)//2} ({progress:.1f}%)")

try:
    # Create animation
    frames = len(data) // 2  # Every 2nd data point
    print(f"Creating animation with {frames} frames...")
    
    anim = FuncAnimation(fig, animate_frame, frames=frames, interval=150, repeat=True, blit=False)
    
    # Save as GIF
    gif_filename = 'hoomd_particle_animation.gif'
    gif_path = os.path.join(current_dir, gif_filename)
    
    print(f"Saving GIF to: {gif_path}")
    print("This may take 1-2 minutes...")
    
    anim.save(gif_path, writer=PillowWriter(fps=8), dpi=100)
    
    # Check if file was created successfully
    if os.path.exists(gif_path):
        file_size_mb = os.path.getsize(gif_path) / (1024 * 1024)
        print(f"\n✓ SUCCESS! GIF created successfully!")
        print(f"  File: {gif_path}")
        print(f"  Size: {file_size_mb:.1f} MB")
        print(f"  Frames: {frames}")
        print(f"  Duration: ~{frames/8:.1f} seconds at 8 FPS")
        print(f"\n🎬 Open '{gif_filename}' in any image viewer to see your animation!")
    else:
        print(f"\n✗ ERROR: GIF file was not created at {gif_path}")

except ImportError:
    print("\n✗ ERROR: Required packages not available.")
    print("Try installing: pip install pillow matplotlib")
    
except Exception as e:
    print(f"\n✗ ERROR creating GIF: {e}")
    print("This might be due to:")
    print("- Insufficient memory (try reducing frames)")
    print("- Missing pillow package (pip install pillow)")
    print("- File permission issues")

finally:
    plt.close('all')  # Clean up

Working directory: /home/sammu/PhD/active-enhancement/docs/summer/case_study_4/active
Loading data...
Loaded 400 frames from file (using first 400 frames)
Creating animation with 200 frames...
Saving GIF to: /home/sammu/PhD/active-enhancement/docs/summer/case_study_4/active/hoomd_particle_animation.gif
This may take 1-2 minutes...
Rendering frame 1/200 (0.2%)
Rendering frame 1/200 (0.2%)
Rendering frame 11/200 (5.2%)
Rendering frame 21/200 (10.2%)
Rendering frame 31/200 (15.2%)
Rendering frame 41/200 (20.2%)
Rendering frame 51/200 (25.2%)
Rendering frame 61/200 (30.2%)
Rendering frame 71/200 (35.2%)
Rendering frame 81/200 (40.2%)
Rendering frame 91/200 (45.2%)
Rendering frame 101/200 (50.2%)
Rendering frame 111/200 (55.2%)
Rendering frame 121/200 (60.2%)
Rendering frame 131/200 (65.2%)
Rendering frame 141/200 (70.2%)
Rendering frame 151/200 (75.2%)
Rendering frame 161/200 (80.2%)
Rendering frame 171/200 (85.2%)
Rendering frame 181/200 (90.2%)
Rendering frame 191/200 (95.2%)
